<a href="https://colab.research.google.com/github/pengyk/COMP551/blob/main/COMP551_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files
from datetime import datetime, timedelta
from dateutil.parser import parse


base_path = "drive/My Drive/COMP551/assignment1/"
search_pattern_path = base_path + "2020_US_weekly_symptoms_dataset.csv"
cases_path = base_path + "aggregated_cc_by.csv"
combined_path = base_path + "combined_data.csv"

# Function to trim off rows and columns with too many NaN

In [ ]:
def clean_data(df):
  (rows, cols) = df.shape
  print(rows, cols)
  row_threshold = int(rows * 0.05)
  col_threshold = int(cols * 0.05)

  df = df.dropna(axis = 0, thresh = row_threshold)
  df = df.dropna(axis = 1, thresh = col_threshold)
  df = df.reset_index(drop = True)
  (rows, cols) = df.shape
  print(rows, cols)
  return df


# Function to only get the data from United States

In [ ]:
def select_USA(df):
  df = df.loc[df['open_covid_region_code'] == 'USA']
  df = df.reset_index(drop = True)
  return df

# Function to transform columns to objects under a "search" column (easier to manipulate later when separating between search and cases)

In [ ]:
def shorten_weekly_to_objects(df):
  df['search'] = ""
  for index, row in df.iterrows():
    row_object = row
    df['search'][index] = row_object

  df = df.loc[:, df.columns.intersection(['date','search'])]
  index = 0
  while index < df.shape[0]:
    curr_date = df['date'][index]
    selected_rows = df.loc[df['date'] == curr_date]
    df['search'][index] = selected_rows
    index += 1
  return df

In [ ]:
def merge_daily_into_weekly(daily_df, weekly_df):
  weekly_df['cases'] = ""
  daily_index = 0
  weekly_index = 0
  # don't want values outside of the range of date - 6 days
  firstday = parse(weekly_df['date'][weekly_index])
  print(firstday)
  six_days_before_first_date = firstday - timedelta(days=6)
  print(six_days_before_first_date)

  # catch up the daily entries up to one week before weekly entries
  while daily_index < daily_df.shape[0] and parse(daily_df['date'][daily_index]) < six_days_before_first_date:
    daily_index += 1

  # catch up the weekly to the daily entries if they are behind
  while weekly_index < weekly_df.shape[0] and parse(weekly_df['date'][weekly_index]) < parse(daily_df['date'][daily_index]):
    weekly_index += 1 

  while daily_index < daily_df.shape[0] and weekly_index < weekly_df.shape[0]:
    arr = []
    while parse(daily_df['date'][daily_index]) <= parse(weekly_df['date'][weekly_index]):
      arr.append(daily_df.iloc[ daily_index , : ])
      daily_index += 1
    weekly_df['cases'][weekly_index] = arr
    weekly_index += 1
  return weekly_df

# Main

In [ ]:
search_df = pd.read_csv(search_pattern_path) 
cases_df = pd.read_csv(cases_path) 

print(type(search_df))

cases_df = select_USA(cases_df)

search_df = clean_data(search_df)
cases_df = clean_data(cases_df)
search_df = shorten_weekly_to_objects(search_df)
print(search_df)
print("____________________")
print(cases_df)
combined_df = merge_daily_into_weekly(cases_df, search_df)
combined_df.head()
print(combined_df)
combined_df.to_csv(combined_path)